In [60]:
# 데이터 분석을 위한 pandas, 수치계산을 위한 numpy, 시각화를 위한 seaborn, matplotlib 을 로드합니다.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [70]:
df_train = pd.read_csv('C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/data/train_df_1009_v2.csv')
df_test = pd.read_csv('C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/data/test_df_1009_v2.csv')
submission = pd.read_csv('C:/Users/KimDongyoung/Desktop/GBT해커톤/GBT_hackerton/data/sample_submission.csv')

In [71]:
df_train.columns

Index(['ID', '분류', '제목', '키워드'], dtype='object')

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer 

## TD-IDF 벡터화

In [72]:
tfidfv = TfidfVectorizer(
                             min_df=10, 
                             )

tfidfv.fit(df_train['키워드'])

TfidfVectorizer(min_df=10)

In [73]:
# 변환
x_train = tfidfv.transform(df_train['키워드'])

In [75]:
X_test = tfidfv.transform(df_test['키워드'])

x_train.shape, X_test.shape

((54314, 32896), (23405, 32896))

In [76]:
# 단어 사전
feature_names = tfidfv.get_feature_names_out()
print(len(feature_names))
feature_names[:10]

32896


array(['100데시벨', '100위', '1012구간', '101동', '102동', '10가지', '10언더', '10위',
       '10필지', '1120위'], dtype=object)

In [69]:
from scipy.sparse import csr_matrix

In [77]:
# 희소 행렬을 사용하여 TF-IDF 값을 단어별로 더합니다.
df_vocab = pd.DataFrame.sparse.from_spmatrix(x_train, columns=feature_names)
df_vocab = df_vocab.sum(axis=0).to_frame(name='sum')
df_vocab

,sum
100데시벨,1.627176
100위,0.881983
1012구간,1.071675
101동,0.901083
102동,0.993293
...,...
힐튼,3.020931
힐튼호텔,1.516208
힘겨루기,0.803744
힘스,2.416833


In [78]:
# np.sum 으로 위에서 구한 train_feature_vector 의 값을 모두 더합니다. axis=0 으로 합니다. 
dist = np.sum(x_train, axis=0)

vocab_count = pd.DataFrame(dist, columns=feature_names)
vocab_count

,100데시벨,100위,1012구간,101동,102동,10가지,10언더,10위,10필지,1120위,...,힐스테이트,힐스테이트레이크송도,힐스테이트몬테로이,힐스테이트용인고진역,힐스테이트용인둔전역,힐튼,힐튼호텔,힘겨루기,힘스,힙합
0,1.627176,0.881983,1.071675,0.901083,0.993293,1.263416,1.344293,8.08336,1.299569,1.499721,...,103.521398,1.809898,1.125406,3.722411,3.011163,3.020931,1.516208,0.803744,2.416833,3.051649


In [79]:
vocab_count.T[0].sort_values(ascending=False).head(30)

반도체    1255.595512
사업     1131.055094
조성      753.316785
도시      699.561386
산업      665.719292
확진자     664.759616
기업      640.742379
단지      625.294796
시설      594.822229
교육      588.078208
추진      572.632889
아파트     551.462066
센터      525.016834
설치      503.775650
선정      501.238555
의원      492.399340
처인구     490.875751
조사      476.779067
안전      475.969889
발생      470.891787
대회      469.999112
주민      458.366994
주택      454.651030
가구      451.306255
경찰      451.078466
신청      447.642227
환경      425.424194
개선      424.914509
규모      419.030611
이용      417.611227
Name: 0, dtype: float64

## 벡터화 데이터를 다시 train, test로 나눠준다

In [63]:
# vectorized_train_x = vectorized[:len(df_train)]
# vectorized_test_x = vectorized[len(df_train):]

## target 값 레이블 인코딩

In [64]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the '키워드' column
df_train['키워드_encoded'] = label_encoder.fit_transform(df_train['키워드'])

# Display the first few rows to verify
df_train[['키워드', '키워드_encoded']].head()

,키워드,키워드_encoded
0,용인문화재단 인문학 콘서트 뮤지컬 이야기 개최 인문학 콘서트 뮤지컬 이야기 용인문화...,30184
1,농촌 테마파크 단체 체험객 체험료 체험일 기준 용인시통합예약사이트 접수 용인특례시 ...,10424
2,노후 주택 에너지 성능 개선 신청 연장 용인시청 노후 건축물 환경친화적 개선 온실 ...,10109
3,수원 고양시 특례시 지정 도시경쟁력 증가 도내 인구 수원 고양 특례시 지정 골자 지...,23394
4,스페인 미국 국제 명예 자문관 위촉 역할 대외 홍보 권태면 주코스타리카 대사 이정관...,24379


In [65]:
# y_train 정의
y_train = df_train['키워드_encoded']

## 로지스틱 회귀

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score, log_loss

In [68]:
# 학습 데이터와 검증 데이터를 나눕니다.
X_train, X_val, y_train_split, y_val = train_test_split(vectorized_train_x, y_train, test_size=0.2, random_state=42)

# 로지스틱 회귀 모델을 초기화합니다.
model = LogisticRegression(max_iter=1000)

# 모델을 학습시킵니다.
model.fit(X_train, y_train_split)

# 검증 데이터로 예측을 수행합니다.
y_val_pred = model.predict(X_val)

# 검증 데이터에 대한 macro F1 score를 계산합니다.
macro_f1 = f1_score(y_val, y_val_pred, average='macro')

# 검증 데이터에 대한 예측 확률을 계산합니다.
y_val_pred_proba = model.predict_proba(X_val)

# 검증 데이터에 대한 log loss를 계산합니다.
loss = log_loss(y_val, y_val_pred_proba)

# 결과를 출력합니다.
print(f'Macro F1 Score: {macro_f1:.4f}')
print(f'Log Loss: {loss:.4f}')

MemoryError: Unable to allocate 13.7 GiB for an array with shape (43351, 42272) and data type float64

In [ ]:
# 테스트 데이터에 대한 예측을 수행합니다.
y_test_pred = model.predict(vectorized_test_x)

# 예측 결과를 출력합니다.
print(y_test_pred)